In [1]:
import pandas as pd
import numpy as np
#from movienet import MovieNet

In [ ]:
from autoencoder import *

In [ ]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [ ]:
ratings=pd.read_csv('ratings1.csv')

In [ ]:
ratings.head()

In [ ]:
ratings.shape

In [ ]:
print(f"full dataset:\nnumber users {int(ratings.userId.nunique())}, number movies {int(ratings.movieId.nunique())}")

In [ ]:
u_uniq = ratings.userId.unique()
user2idx = {o:i for i,o in enumerate(u_uniq)}
ratings.userId = ratings.userId.apply(lambda x: user2idx[x])

m_uniq = ratings.movieId.unique()
movie2idx = {o:i for i,o in enumerate(m_uniq)}
ratings.movieId = ratings.movieId.apply(lambda x: movie2idx[x])

In [ ]:
n_users=int(ratings.userId.nunique())
n_movies=int(ratings.movieId.nunique())
n_factors = 100
print(f"number users: {n_users}, number movies {n_movies}")

In [ ]:
min_rating,max_rating = ratings.rating.min(),ratings.rating.max()


In [ ]:
x = ratings.drop(['rating', 'timestamp'],axis=1)
y = ratings['rating'].astype(np.float32)

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
ratings_final=ratings.drop(['timestamp'],axis=1)
ratings_final=ratings_final.astype({"rating": np.float32})
ratings_final.head()

In [ ]:
def dataPreprocessor(rating_df, num_users, num_items, init_value=0, average=False):
    """
        INPUT: 
            data: pandas DataFrame. columns=['userID', 'itemID', 'rating' ...]
            num_row: int. number of users
            num_col: int. number of items
            
        OUTPUT:
            matrix: 2D numpy array. 
    """
    matrix = np.zeros((num_users, num_items),dtype='float32')
    for (_, userID, itemID, rating) in rating_df.itertuples():
        matrix[userID, itemID] = rating

    return matrix

In [ ]:
new_data=dataPreprocessor(ratings_final, n_users,n_movies)

In [ ]:
ratings_final=ratings_final.astype({"rating": np.float32})

In [ ]:
new_data

In [ ]:
new_df=pd.DataFrame(new_data)

In [ ]:
new_df.head()

In [ ]:
new_df.to_pickle('new_df_matrix.pkl')

In [ ]:
ae = AutoEncoder(new_df, validation_perc=0.05, lr=0.5, intermediate_size=50, encoded_size=10)


In [ ]:
ae.train_loop(epochs=20)

In [ ]:
losses = pd.DataFrame(data=list(zip(ae.train_losses, ae.val_losses)), columns=['train_loss', 'validation_loss'])
losses['epoch'] = (losses.index + 1) / 3

In [ ]:
fig, ax = plt.subplots()
ax.plot(losses['epoch'], losses['train_loss'])
ax.plot(losses['epoch'], losses['validation_loss'])
ax.set_ylabel('MSE loss')
ax.set_xlabel('epoch')
ax.set_title('autoencoder loss over time')
ax.legend()

In [ ]:
encoded = ae.get_encoded_representations()

In [ ]:
encoded.shape


In [ ]:
encoded

In [ ]:
y=pd.read_csv('movies.csv')
y.head()

In [91]:
#collaborative_embeddings = pd.read_pickle("autoencoder_embeddings (1).pkl")
#collaborative_embeddings = pd.read_pickle("autoencodercoll42_embeddings.pkl")
#collaborative_embeddings = pd.read_pickle("autoencodercoll_embeddings2.pkl")
collaborative_embeddings = pd.read_pickle("autoencodercolln_embeddings.pkl")
print(collaborative_embeddings.shape)
collaborative_embeddings= pd.DataFrame(collaborative_embeddings)
collaborative_embeddings.head()

(10677, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.412255,0.0,0.000000,0.0,0.0,0.154076,0.000000,0.257120,...,1.673705,0.0,0.0,0.0,1.197119,0.0,0.0,0.000000,2.323226,0.0
1,0.0,0.0,1.543272,0.0,2.818511,0.0,0.0,0.000000,0.703664,2.160406,...,5.061376,0.0,0.0,0.0,4.918608,0.0,0.0,0.000000,2.754534,0.0
2,0.0,0.0,4.433138,0.0,4.809015,0.0,0.0,0.000000,0.000000,0.000000,...,9.035736,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
3,0.0,0.0,0.567439,0.0,3.398583,0.0,0.0,0.000000,1.712569,3.404999,...,6.966730,0.0,0.0,0.0,6.296949,0.0,0.0,2.191186,3.016886,0.0
4,0.0,0.0,0.000000,0.0,4.544496,0.0,0.0,0.000000,6.380806,2.665438,...,5.008911,0.0,0.0,0.0,9.404757,0.0,0.0,0.000000,0.000000,0.0


In [62]:
#content_embeddings = pd.read_pickle("autoencodercon_embeddings_3.pkl")
content_embeddings = pd.read_pickle("autoencodercont3_embeddings.pkl")
print(content_embeddings.shape)
content_embeddings= pd.DataFrame(content_embeddings)
content_embeddings.head()

(10677, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.924879,0.000000,0.006536,0.0,0.000000,0.0,...,0.732923,0.0,0.073787,0.0,0.000000,0.135765,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.000000,0.132794,0.356823,0.0,0.033746,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.201291,0.0,0.0,0.0,0.549948
2,0.0,0.0,0.0,0.0,0.070235,0.000000,0.075572,0.0,0.000000,0.0,...,0.144563,0.0,0.031774,0.0,0.050217,0.078915,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.003593,0.000000,0.024650,0.0,0.000000,0.0,...,0.231564,0.0,0.085713,0.0,0.204439,0.108164,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.054179,0.000000,0.066488,0.0,0.000000,0.0,...,0.014557,0.0,0.000000,0.0,0.293759,0.087323,0.0,0.0,0.0,0.000000


In [87]:
hy_embeddings = pd.read_pickle("autoencoderhy_embeddings.pkl")
print(hy_embeddings.shape)
hy_embeddings= pd.DataFrame(hy_embeddings)
hy_embeddings.head()

(10677, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.120069,1.386228,0.0,0.0,0.0,0.0,0.0,0.283937,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.656815
1,0.0,5.585257,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.629687,0.0,0.0,0.0,0.000000
2,0.0,0.766058,5.320053,0.0,0.0,0.0,0.0,0.0,8.645866,0.0,...,0.0,0.0,0.0,0.0,0.0,3.871958,0.0,0.0,0.0,5.889831
3,0.0,6.671350,1.338027,0.0,0.0,0.0,0.0,0.0,2.266763,0.0,...,0.0,0.0,0.0,0.0,0.0,3.361939,0.0,0.0,0.0,0.000000
4,0.0,9.505838,4.597833,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,4.319620,0.0,0.0,0.0,0.716826


In [16]:
movies = pd.read_csv("movies1.dat", sep='::', header=None, engine='python', encoding='latin-1')
movies.columns=['movieId','title','genres']
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
ratings = pd.read_csv("ratings1.dat", sep='::', header=None, engine='python', encoding='latin-1')
ratings.columns=['userId','movieId', 'rating', 'timestamp']
ratings.head()

,userId,movieId,rating,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392


In [18]:
m_uniq = ratings.movieId.unique()
movie2idx = {o:i for i,o in enumerate(m_uniq)}

In [19]:
#movies = pd.read_csv("movies.dat", sep='::', header=None, engine='python', encoding='latin-1')
print("{} unique movies in movies.csv".format(len(movies.movieId.unique())))

#ratings = pd.read_csv("ratings.dat", sep='::', header=None, engine='python', encoding='latin-1')
print("{} unique movies in ratings.csv".format(len(ratings.movieId.unique())))

movies = pd.merge(movies, ratings, on="movieId", how="inner")
movies.movieId = movies.movieId.apply(lambda x: movie2idx[x])

#get popularity
popularity = pd.DataFrame(movies[['userId', 'title', 'movieId']].groupby(['title', 'movieId']).agg(['count']))
popularity.reset_index(inplace=True)
popularity.columns = ['title', 'movieId', 'ratings_count']
popularity.sort_values('ratings_count', ascending=False, inplace=True)
movies = pd.merge(popularity[['movieId', 'ratings_count']], movies, on='movieId')
movies.reset_index(inplace=True)

#get average ratings
average_ratings = pd.DataFrame(movies[['rating', 'title', 'movieId']].groupby(['title', 'movieId']).agg(['mean']))
average_ratings.reset_index(inplace=True)
average_ratings.columns = ['title', 'movieId', 'avg_rating']
movies = pd.merge(average_ratings[['movieId', 'avg_rating']], movies, on='movieId')
movies.reset_index(inplace=True)


movies = movies[['movieId', 'title', 'genres', 'ratings_count', 'avg_rating']]
movies.drop_duplicates(inplace=True)
print("{} unique movies in embeddings".format(len(movies.movieId.unique())))
movies.set_index('movieId', inplace=True, drop=True)
movies.sort_index(ascending=True, inplace=True)
print(movies.shape)
movies.head(5)

10681 unique movies in movies.csv
10677 unique movies in ratings.csv
10677 unique movies in embeddings
(10677, 4)


,title,genres,ratings_count,avg_rating
movieId,,,,
0,Boomerang (1992),Comedy|Romance,2412,2.861318
1,"Net, The (1995)",Action|Crime|Thriller,14975,3.125209
2,Dumb & Dumber (1994),Comedy,17851,2.936950
3,Outbreak (1995),Action|Drama|Sci-Fi|Thriller,16075,3.418414
4,Stargate (1994),Action|Adventure|Sci-Fi,18925,3.349353


In [20]:
from similarity import SimilarityPredictions

In [21]:
def lookup_movie_id_by_title(movie_title):
    return movies[movies.title.str.contains(movie_title)]

In [276]:
lookup_movie_id_by_title("Dead")

,title,genres,ratings_count,avg_rating
movieId,,,,
43,Dead Poets Society (1989),Drama,10604,3.887873
264,Dead Again (1991),Mystery|Romance|Thriller,2388,3.749581
307,Dead Man Walking (1995),Crime|Drama,12689,3.957680
330,"Quick and the Dead, The (1995)",Action|Adventure|Western,3979,3.040839
396,Drop Dead Fred (1991),Comedy|Fantasy,1848,2.578193
409,Evil Dead II (Dead by Dawn) (1987),Action|Comedy|Fantasy|Horror,3952,3.792637
493,Freddy's Dead: The Final Nightmare (1991),Horror,808,2.217203
618,Bringing Out the Dead (1999),Drama|Horror,2190,3.140183
662,Waking the Dead (2000),Drama|Thriller,360,3.104167


In [23]:
cind=1450
lotr=798
note=67

In [281]:
#sm=4929
#sm=3363
#sm=1543
sm=724
#sm=618

In [11]:
def get_detailed_recs(movie_id, embeddings, file_path):
    #get similar movies
    sim_model = SimilarityPredictions(embeddings, similarity_metric="cosine")
    output = sim_model.predict_similar_items(seed_item=movie_id, n=20)
    similar_movies = pd.DataFrame(output)
    similar_movies.set_index('item_id', inplace=True)
    sim_df = pd.merge(movies, similar_movies, left_index=True, right_index=True)
    sim_df.sort_values('similarity_score', ascending=False, inplace=True)
    
    #save recs locally
    sim_df.head(20).to_csv(file_path, index=False, header=True)
    return sim_df.head(20)

In [282]:
get_detailed_recs(sm, content_embeddings, 'content_recs_cind.csv')

,title,genres,ratings_count,avg_rating,similarity_score
724,"Evil Dead, The (1981)",Fantasy|Horror,2532,3.661137,1.000000
409,Evil Dead II (Dead by Dawn) (1987),Action|Comedy|Fantasy|Horror,3952,3.792637,0.846004
1610,Night of the Living Dead (1968),Horror|Sci-Fi|Thriller,4202,3.628986,0.836847
3034,"Mummy, The (1959)",Horror,373,3.126005,0.831568
7298,Class of Nuke 'Em High (1986),Comedy|Horror,177,2.276836,0.830757
7774,Citizen Toxie: The Toxic Avenger IV (2000),Action|Comedy|Horror,36,3.027778,0.813691
1155,"Toxic Avenger, The (1985)",Comedy|Horror,654,2.724006,0.811526
3666,Dead Alive (Braindead) (1992),Comedy|Fantasy|Horror,1284,3.671729,0.807302
977,Resident Evil: Apocalypse (2004),Action|Horror|Sci-Fi|Thriller,1008,2.879960,0.799041
4180,Quills (2000),Drama|Thriller,1516,3.515501,0.797696


In [283]:
get_detailed_recs(sm, collaborative_embeddings, 'collaborative_recs_cind.csv')

,title,genres,ratings_count,avg_rating,similarity_score
724,"Evil Dead, The (1981)",Fantasy|Horror,2532,3.661137,1.000000
409,Evil Dead II (Dead by Dawn) (1987),Action|Comedy|Fantasy|Horror,3952,3.792637,0.922419
3343,Dawn of the Dead (1978),Action|Drama|Horror,1223,3.800899,0.916275
2307,Ring (Ringu) (1998),Horror|Mystery|Thriller,1301,3.617602,0.914258
2072,Hellraiser (1987),Horror,1309,3.294500,0.913229
3666,Dead Alive (Braindead) (1992),Comedy|Fantasy|Horror,1284,3.671729,0.911045
3713,Tremors (1990),Comedy|Horror|Sci-Fi,1615,3.328793,0.893879
405,Army of Darkness (1993),Action|Adventure|Comedy|Fantasy|Horror,6768,3.722370,0.888798
3716,Day of the Dead (1985),Horror|Thriller,502,3.446215,0.888200
1315,Bad Taste (1987),Comedy|Horror|Sci-Fi,1050,3.508095,0.885895


In [284]:
get_detailed_recs(sm, hy_embeddings, 'hy_recs_cind.csv')

,title,genres,ratings_count,avg_rating,similarity_score
724,"Evil Dead, The (1981)",Fantasy|Horror,2532,3.661137,1.000000
3718,Bubba Ho-tep (2002),Comedy|Horror,1004,3.555279,0.902878
2072,Hellraiser (1987),Horror,1309,3.294500,0.893269
409,Evil Dead II (Dead by Dawn) (1987),Action|Comedy|Fantasy|Horror,3952,3.792637,0.881886
5035,"Return of the Living Dead, The (1985)",Comedy|Horror|Sci-Fi,560,3.289286,0.878855
3713,Tremors (1990),Comedy|Horror|Sci-Fi,1615,3.328793,0.876206
909,Darkman (1990),Action|Crime|Horror|Sci-Fi|Thriller,673,3.071322,0.862098
768,Meet the Feebles (1989),Animation|Comedy|Musical,463,3.353132,0.862033
3343,Dawn of the Dead (1978),Action|Drama|Horror,1223,3.800899,0.860932
3666,Dead Alive (Braindead) (1992),Comedy|Fantasy|Horror,1284,3.671729,0.860750
